In [1]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset
import pandas as pd
import torchaudio
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
from torchsummary import summary
import optuna

In [2]:
class EmotionDataset(Dataset):
    def __init__(self, data_path, final_sample_rate, num_samples_limit, transformation, device):
        
        self.data_path = data_path
        
        self.final_sample_rate = final_sample_rate
        self.num_samples_limit = num_samples_limit
        self.device = device
        self.transformation = transformation.to(self.device)
        self.df = pd.read_csv(data_path)
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        audio_path = self.get_audio_path(index)
        label = self.get_label(index)
        signal, sr = torchaudio.load(audio_path)
        
        signal = signal.to(self.device)
        signal = self.to_mono_if_necessary(signal)
        signal = self.resample_if_necessary(signal, sr)
        signal = self.cut_down_if_necessary(signal)
        signal = self.right_pad_if_necessary(signal)
        signal = self.transformation(signal)
        
        signal[signal == 0] = 0.000000001
        signal = torch.log(signal)
        
        return signal, label
    
    def resample_if_necessary(self, signal, original_sr):
        if original_sr != self.final_sample_rate:
            resampler = torchaudio.transforms.Resample(original_sr, self.final_sample_rate).to(self.device)
            signal = resampler(signal)
        
        return signal
    
    def to_mono_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True).to(self.device)
        return signal
    
    def cut_down_if_necessary(self, signal):
        return signal[:, :self.num_samples_limit].to(self.device)
    
    def right_pad_if_necessary(self, signal):
        if signal.shape[1] < self.num_samples_limit:
            num_missing_samples = self.num_samples_limit - signal.shape[1]
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding).to(self.device)
        return signal
        
    def get_audio_path(self, index):
        return '../' + self.df.iloc[index].filename
    
    def get_label(self, index):
        return self.df.iloc[index].category_num
    
    def plot_spectrogram(self, index, log=True):
        spec = self.__getitem__(index)[0]
        spec = spec.cpu().numpy().reshape(spec.shape[1], -1)
        frame_rate = self.final_sample_rate / self.transformation.hop_length
        seconds = np.arange(spec.shape[1]) / frame_rate
        
        if not log:
            spec = np.exp(spec)
        
        plt.figure(figsize=(10, 6))
        plt.imshow(spec, cmap='inferno', origin='lower', aspect='auto')
        plt.xlabel('Time (s)')
        plt.ylabel('Mel')
        plt.colorbar(label='Intensity (dB)')
        plt.xticks(np.linspace(0, spec.shape[1], 12)[:-1], np.linspace(0, seconds[-1], 12).round(2)[:-1])
        plt.show()

In [3]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [4]:
class CNNNetwork(nn.Module):
    def __init__(self, trial, loss_fn, device, train_data_loader, validate_data_loader):
        super().__init__()
        
        self.num_layers = trial.suggest_int('num_layers', 3, 8)
        self.learning_rate = trial.suggest_categorical("learning_rate", [0.01, 0.005, 0.001, 0.0005, 0.0001])
        self.loss_fn = loss_fn
        self.device = device
        self.train_data_loader = train_data_loader
        self.validate_data_loader = validate_data_loader
        self.layers = nn.ModuleList()
        self.epoch_train_losses = []
        self.epoch_valid_losses = []
        
        dropout_rate = trial.suggest_float('dropout_rate', 0, 0.5)
        
        self.layers = nn.ModuleList()
        in_kernels = 1
        out_kernels = 16
        for i in range(self.num_layers):
            conv = nn.Sequential(
                nn.Conv2d(in_channels=in_kernels, out_channels=out_kernels, kernel_size=3, stride=1, padding=2),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2),
                nn.Dropout2d(dropout_rate)
            )
            self.layers.append(conv)
            
            in_kernels = out_kernels
            out_kernels *= 2
        
        self.flatten = nn.Flatten()
        self.linear = nn.LazyLinear(6)
        self.softmax = nn.Softmax(dim=1)
        
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, 5)
        scheduler = trial.suggest_categorical('scheduler', ['cos', 'exp'])
        
        if scheduler == 'exp':
            self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.9)
    
    def forward(self, input_data):
        out = self.layers[0](input_data)
        
        for layer in self.layers[1:]:
            out = layer(out)
        
        out = self.flatten(out)
        out = self.linear(out)
        predictions = self.softmax(out)
        
        return predictions
    
    def train(self, num_epochs):
        self.epoch_losses = []
        early_stopper = EarlyStopper(patience=3, min_delta=0)
        
        for i in range(1, num_epochs+1):
            print(f'Epoch {i} / {num_epochs} started')
            train_loss = self.__train_single_epoch()
            self.epoch_train_losses.append(train_loss)
            
            valid_loss = self.__validate_single_epoch()
            self.epoch_valid_losses.append(valid_loss)
            
            if early_stopper.early_stop(valid_loss):             
                print(f'Training stopped at Epoch {i} due to early stopping critera')
                break
            
            self.scheduler.step()
            print(f'Epoch {i} / {num_epochs} finished')
            print()
            
        return self.epoch_valid_losses[-1]
    
    def __train_single_epoch(self):
        for inp, target in self.train_data_loader:
            inp, target = inp.to(self.device), target.to(self.device)

            pred = self(inp)
            loss = self.loss_fn(pred, target)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        print(f'Train Loss = {loss.item()}')
        return float(loss.item())
    
    def __validate_single_epoch(self):
        total_loss, n = 0, 0
        
        with torch.no_grad():
            for inp, target in self.validate_data_loader:
                n += 1
                inp, target = inp.to(self.device), target.to(self.device)

                pred = self(inp)
                total_loss += self.loss_fn(pred, target).item()
            
        print(f'Validation Loss = {total_loss / n}')
        return total_loss / n
    
    def predict(self, inp, target, class_mapping):
        #self.eval()
    
        with torch.no_grad():
            prediction_probs = self(inp)
            predicted_index = prediction_probs[0].argmax()
            prediction = class_mapping[predicted_index]
            expected = class_mapping[target]
    
        return prediction, expected
    
    def get_train_epoch_losses(self):
        return self.epoch_train_losses
    
    def get_valid_epoch_losses(self):
        return self.epoch_valid_losses

In [5]:
def create_data_loader(train_data, batch_size):
    data_loader = DataLoader(train_data, batch_size = batch_size)
    return data_loader

def predict(model, inp, target, class_mapping):
    model.eval()
    
    with torch.no_grad():
        prediction_probs = model(inp)
        predicted_index = prediction_probs[0].argmax()
        prediction = class_mapping[predicted_index]
        expected = class_mapping[target]
    
    return prediction, expected

In [6]:
RECORDING_DF_PATH = os.path.join('..', 'data', 'recording_paths.csv')
RECORDING_TRAIN_PATH = os.path.join('..', 'data', 'recording_paths_train.csv')
RECORDING_TEST_PATH = os.path.join('..', 'data', 'recording_paths_test.csv')

FINAL_SR = 16000
NUM_SAMPLES_LIMIT = 60000
BATCH_SIZE = 128

mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=FINAL_SR,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [7]:
def objective(trial):
    emd_train = EmotionDataset(RECORDING_TRAIN_PATH, FINAL_SR, NUM_SAMPLES_LIMIT, mel_spectrogram, device)
    emd_valid = EmotionDataset(RECORDING_TEST_PATH, FINAL_SR, NUM_SAMPLES_LIMIT, mel_spectrogram, device)
    
    train_dataloader = create_data_loader(emd_train, BATCH_SIZE)
    valid_dataloader = create_data_loader(emd_valid, BATCH_SIZE)
    
    loss_fn = nn.CrossEntropyLoss()
    
    model = CNNNetwork(trial,
                 loss_fn=loss_fn,
                 device=device,
                 train_data_loader=train_dataloader,
                 validate_data_loader=valid_dataloader
                ).to(device)
    
    num_epochs = 25
    
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
        
    return model.train(num_epochs=num_epochs)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50);

[I 2024-05-10 07:27:20,316] A new study created in memory with name: no-name-eb4f60d5-abe0-4e56-9240-2d91d81e4629
/opt/conda/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Epoch 1 / 25 started
Train Loss = 1.7891125679016113
Validation Loss = 1.78812970717748
Epoch 1 / 25 finished

Epoch 2 / 25 started
Train Loss = 1.7904282808303833
Validation Loss = 1.7816545764605205
Epoch 2 / 25 finished

Epoch 3 / 25 started
Train Loss = 1.7335582971572876
Validation Loss = 1.7508890628814697
Epoch 3 / 25 finished

Epoch 4 / 25 started
Train Loss = 1.7550510168075562
Validation Loss = 1.7325004438559215
Epoch 4 / 25 finished

Epoch 5 / 25 started
Train Loss = 1.6857129335403442
Validation Loss = 1.7248924175898235
Epoch 5 / 25 finished

Epoch 6 / 25 started
Train Loss = 1.7127782106399536
Validation Loss = 1.7323412994543712
Epoch 6 / 25 finished

Epoch 7 / 25 started
Train Loss = 1.7406210899353027
Validation Loss = 1.7261556486288707
Epoch 7 / 25 finished

Epoch 8 / 25 started
Train Loss = 1.7089612483978271
Validation Loss = 1.7140463491280873
Epoch 8 / 25 finished

Epoch 9 / 25 started
Train Loss = 1.7181434631347656
Validation Loss = 1.6936624149481456
Epoch 9 

[I 2024-05-10 07:46:52,264] Trial 0 finished with value: 1.611531009276708 and parameters: {'num_layers': 6, 'learning_rate': 0.0001, 'dropout_rate': 0.40040728456613145, 'scheduler': 'cos'}. Best is trial 0 with value: 1.611531009276708.


Validation Loss = 1.611531009276708
Epoch 25 / 25 finished

Epoch 1 / 25 started


/opt/conda/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


...

Train Loss = 1.7732919454574585
Validation Loss = 1.7697990536689758
Epoch 1 / 25 finished

Epoch 2 / 25 started
Train Loss = 1.684875249862671
Validation Loss = 1.696923742691676
Epoch 2 / 25 finished

Epoch 3 / 25 started
Train Loss = 1.6642383337020874
Validation Loss = 1.6895921230316162
Epoch 3 / 25 finished

Epoch 4 / 25 started
Train Loss = 1.6731377840042114
Validation Loss = 1.675562580426534
Epoch 4 / 25 finished

Epoch 5 / 25 started
Train Loss = 1.6459057331085205
Validation Loss = 1.6727959016958873
Epoch 5 / 25 finished

Epoch 6 / 25 started
Train Loss = 1.6469078063964844
Validation Loss = 1.6822051803270976
Epoch 6 / 25 finished

Epoch 7 / 25 started
Train Loss = 1.5852121114730835
Validation Loss = 1.650703380505244
Epoch 7 / 25 finished

Epoch 8 / 25 started
Train Loss = 1.6493793725967407
Validation Loss = 1.6551845868428547
Epoch 8 / 25 finished

Epoch 9 / 25 started
Train Loss = 1.636783242225647
Validation Loss = 1.6579329470793407
Epoch 9 / 25 finished

Epoch 10 

[I 2024-05-10 20:30:34,270] Trial 48 finished with value: 1.701959639787674 and parameters: {'num_layers': 8, 'learning_rate': 0.001, 'dropout_rate': 0.4960878593441721, 'scheduler': 'cos'}. Best is trial 45 with value: 1.4806301593780518.


Validation Loss = 1.701959639787674
Training stopped at Epoch 10 due to early stopping critera
Epoch 1 / 25 started


/opt/conda/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Train Loss = 1.6236953735351562
Train Loss = 1.4951506853103638
Validation Loss = 1.5701002379258473
Epoch 13 / 25 finished

Epoch 14 / 25 started


In [ ]:
emd_train = EmotionDataset(RECORDING_TRAIN_PATH, FINAL_SR, NUM_SAMPLES_LIMIT, mel_spectrogram, device)